In [1]:
%pip install scanpy -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scanpy as sc

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
!curl -O https://covid19.cog.sanger.ac.uk/submissions/release1/haniffa21.processed.h5ad

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6854M  100 6854M    0     0  38.2M      0  0:02:58  0:02:58 --:--:-- 39.2M 0  36.9M      0  0:03:05  0:00:27  0:02:38 39.1M02:59  0:02:16  0:00:43 37.8M


In [4]:
data = sc.read('haniffa21.processed.h5ad')
data

/opt/conda/lib/python3.10/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


AnnData object with n_obs × n_vars = 647366 × 24929
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id'
    var: 'feature_types'
    uns: 'hvg', 'leiden', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    layers: 'raw'

In [28]:
data.obs

,sample_id,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,full_clustering,initial_clustering,Resample,Collection_Day,...,Status,Smoker,Status_on_day_collection,Status_on_day_collection_summary,Days_from_onset,Site,time_after_LPS,Worst_Clinical_Status,Outcome,patient_id
covid_index,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGAAACCTA-MH9179824,MH9179824,1160,1160,2172.0,79.0,3.637201,CD8.TE,CD8,Initial,D0,...,Covid,Non-smoker,Ward_O2,Moderate,8,Ncl,nan,Moderate,Home,MH9179824
AAACCTGAGAGTAATC-MH9179824,MH9179824,1419,1419,3947.0,54.0,1.368128,CD4.IL22,CD4,Initial,D0,...,Covid,Non-smoker,Ward_O2,Moderate,8,Ncl,nan,Moderate,Home,MH9179824
AAACCTGAGAGTGAGA-MH9179824,MH9179824,1986,1986,5886.0,143.0,2.429494,CD8.Naive,CD4,Initial,D0,...,Covid,Non-smoker,Ward_O2,Moderate,8,Ncl,nan,Moderate,Home,MH9179824
AAACCTGAGGAATCGC-MH9179824,MH9179824,1508,1508,4271.0,106.0,2.481854,CD4.Naive,CD4,Initial,D0,...,Covid,Non-smoker,Ward_O2,Moderate,8,Ncl,nan,Moderate,Home,MH9179824
AAACCTGAGTGTTGAA-MH9179824,MH9179824,1109,1109,2279.0,76.0,3.334796,CD8.EM,CD4,Initial,D0,...,Covid,Non-smoker,Ward_O2,Moderate,8,Ncl,nan,Moderate,Home,MH9179824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BGCV15_TTTGTCAAGGCGATAC-1,BGCV15_CV0176,2401,2401,8633.0,490.0,5.675895,CD4.CM,CD4,Resample,D28,...,Covid,Non-smoker,ITU_O2,Severe,31,Cambridge,nan,Critical,unknown,CV0176
BGCV15_TTTGTCACAAGCCATT-1,BGCV15_CV0944,1973,1973,5315.0,103.0,1.937912,NK_16hi,NK_16hi,Initial,D0,...,Healthy,Not_known,Healthy,Healthy,Healthy,Cambridge,nan,Healthy,Home,CV0944
BGCV15_TTTGTCACAGACACTT-1,BGCV15_CV0257,1209,1209,3345.0,230.0,6.875935,CD8.EM,CD8,Resample,D28,...,Covid,Non-smoker,Ward_O2,Moderate,13,Cambridge,nan,Moderate,Home,CV0257


In [27]:
data.var

,feature_types
MIR1302-2HG,Gene Expression
AL627309.1,Gene Expression
AL627309.3,Gene Expression
AL627309.2,Gene Expression
AL669831.2,Gene Expression
...,...
AB_Podocalyxin,Antibody Capture
AB_GGT1,Antibody Capture
AB_c-Met,Antibody Capture
AB_LIGHT,Antibody Capture


In [20]:
(data.var['feature_types'] == "Gene Expression").sum()

24737

In [37]:
def explore_obs(data, tag):
    values, counts = np.unique(data.obs[tag], return_counts=True)
    
    if len(values) > 1000:
        return
    
    print(f"{len(values)} unique {tag} elements")
    print(f"--------------------------------------------")
    
    max_value_len = max([len(str(value)) for value in values])
    max_count_len = max([len(str(count)) for count in counts])
    
    for i, (value, count) in enumerate(reversed(sorted(list(zip(values, counts)), key = lambda x: x[1]))):
        print(f"{value:<{max_value_len+3}} {count:>{max_count_len}}")

In [38]:
for ob in data.obs.columns:
    explore_obs(data, ob)
    print("\n")

143 unique sample_id elements
--------------------------------------------
MH9143427         14317
AP6               14086
MH8919333         12081
MH9143277         11710
AP11              10921
MH9143326         10446
MH9143327         10295
MH9143276         10207
MH9143270         10084
MH9143323         10082
MH9143324         10014
MH9143422          9608
MH9143271          9542
MH9143272          9458
MH9143423          9083
MH9143325          9019
MH9179825          8919
MH8919327          8819
MH9143321          8639
AP12               8574
MH9143424          8530
AP5                8511
MH9143275          8371
MH9143322          8244
newcastle49        8232
MH9179824          8205
MH8919328          8182
MH9143421          8111
MH8919332          8004
MH9143273          7987
MH9143420          7985
MH9143426          7953
MH8919326          7816
MH8919330          7605
MH8919331          7585
AP4                7446
newcastle21v2      7300
MH9143320          7128
MH8919283    

In [12]:
gex_data = data[:, data.var['feature_types']=="GEX"]
atac_data = data[:, data.var['feature_types']=="ATAC"]

print(gex_data)
print()
print(atac_data)

View of AnnData object with n_obs × n_vars = 69249 × 13431
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker'
    var: 'feature_types', 'gene_id'
    uns: 'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
    obsm: 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

View of AnnData object with n_obs × n_vars = 69249 × 116490
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_b

In [248]:
n_best_features = 100

In [249]:
gex_most_measured_features = np.argsort(((gex_data.X > 0).sum(axis=0) / gex_data.X.shape[0]).A)
gex_feature_filtered = gex_data[:, gex_most_measured_features[0, -n_best_features:]]

In [250]:
atac_most_measured_features = np.argsort(((atac_data.X > 0).sum(axis=0) / atac_data.X.shape[0]).A)
atac_feature_filtered = atac_data[:, atac_most_measured_features[0, -n_best_features:]]

In [251]:
(gex_feature_filtered.X > 0).sum(axis=0) / gex_feature_filtered.shape[0]

matrix([[0.55020289, 0.5512715 , 0.55290329, 0.55336539, 0.55476613,
         0.55675894, 0.55724992, 0.55743765, 0.55797196, 0.5580586 ,
         0.55924273, 0.56275181, 0.56553885, 0.56558217, 0.56656414,
         0.5666941 , 0.56819593, 0.56845586, 0.56974108, 0.56981328,
         0.57040535, 0.57082413, 0.57166168, 0.57192162, 0.57310575,
         0.57314907, 0.57365449, 0.57392887, 0.57576283, 0.57636933,
         0.57779896, 0.57932967, 0.58081705, 0.58135136, 0.58296871,
         0.58302647, 0.58536585, 0.5873009 , 0.58809513, 0.58926483,
         0.59160421, 0.59661511, 0.59738047, 0.59876677, 0.59953212,
         0.60060073, 0.60129388, 0.60486072, 0.60806654, 0.6119944 ,
         0.61275975, 0.61602334, 0.61805947, 0.62422562, 0.62445667,
         0.6245722 , 0.62620399, 0.62659389, 0.63176364, 0.63603807,
         0.63654349, 0.63661569, 0.64071683, 0.64074571, 0.64240639,
         0.64474577, 0.64695519, 0.6484859 , 0.64971335, 0.65241375,
         0.65254372, 0.65857991, 0

In [252]:
(atac_feature_filtered.X > 0).sum(axis=0) / atac_feature_filtered.shape[0]

matrix([[0.38942079, 0.38959407, 0.3896374 , 0.38968072, 0.38973848,
         0.38982512, 0.39053272, 0.39083597, 0.39173129, 0.39217895,
         0.39261217, 0.39288654, 0.39290098, 0.39383962, 0.39396959,
         0.39456165, 0.39516816, 0.39539921, 0.39565914, 0.39577467,
         0.39619345, 0.39635229, 0.39638118, 0.3973487 , 0.39755087,
         0.397883  , 0.39886497, 0.39955812, 0.40211411, 0.40221519,
         0.40298055, 0.40309607, 0.40339933, 0.40468454, 0.40478563,
         0.40695172, 0.40744271, 0.40803477, 0.4096088 , 0.40966657,
         0.40982541, 0.41112507, 0.41190487, 0.41298791, 0.41422981,
         0.41431645, 0.41518289, 0.41565943, 0.41629482, 0.41680024,
         0.41693021, 0.41706017, 0.4181721 , 0.41827319, 0.41860532,
         0.41961617, 0.42000607, 0.42046817, 0.42097359, 0.42121908,
         0.42134904, 0.42166674, 0.4217245 , 0.42202775, 0.42322633,
         0.42452599, 0.42537798, 0.42599893, 0.4262011 , 0.42677873,
         0.42683649, 0.42690869, 0

In [284]:
gex_feature_filtered.shape

(69249, 100)

In [285]:
atac_feature_filtered.shape

(69249, 100)

In [254]:
%pip install torchinfo

Note: you may need to restart the kernel to use updated packages.


In [255]:
import torch
import numpy as np
import torch.nn as nn
from torchinfo import summary

In [256]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
device

'cuda'

In [286]:
class SplitActivator(nn.Module):
    def __init__(self, latent_features, mean_act = nn.Tanh(), logvar_act = nn.ReLU()):
        super().__init__()

        self.latent_features = latent_features
        self.mean_act = mean_act
        self.logvar_act = logvar_act
    
    def forward(self, x):
#         print(f"splitinx: {x.shape}")
        mean, logvar = torch.split(x, self.latent_features, dim=-1)
        logvar = torch.clamp(logvar, min=1e-6)

        mean = self.mean_act(mean)
        logvar = self.logvar_act(logvar)
        
#         print(f"postsplit: {mean.shape, logvar.shape}")
        
        return torch.cat((mean, logvar), axis=-1)

In [287]:
class Sampler(nn.Module):

    def __init__(self, latent_features):
        super().__init__()
        self.latent_features = latent_features

    def forward(self, mean, logvar):
        std = torch.exp(logvar*0.5)

        epsilon = torch.normal(torch.zeros(mean.shape), torch.ones(std.shape)).to(device)

        sample = mean + epsilon * std

        return sample


In [318]:
class VarAutoencoder(nn.Module):

    def __init__(self, input_features, latent_features):
        super().__init__()

        self.input_features = input_features
        self.latent_features = latent_features

        hidden_features = 2*latent_features
        self.encoder = nn.Sequential(
            nn.Linear(input_features, hidden_features),
            nn.ReLU(),
            nn.Linear(hidden_features, 2*latent_features),
            SplitActivator(latent_features),
        )

        self.sampler = Sampler(latent_features)

        self.decoder = nn.Sequential(
            nn.Linear(latent_features, hidden_features),
            nn.ReLU(),
            nn.Linear(hidden_features, input_features),
            nn.Sigmoid(),
        )
        
    def encode(self, x, split=True):
        
        x = self.encoder(x)
        
        if split:
            mean, logvar = torch.split(x, self.latent_features, dim=-1)
            return mean, logvar
        
        return x
        
    def embed(self, x, return_params=False):
        mean, logvar = self.encode(x, split=True)
        z = self.sampler(mean, logvar)
        
        if return_params:
            return z, mean, logvar
        
        return z

    def forward(self, x):
        
#         x = self.encoder(x)
#         mean, logvar = torch.split(x, self.latent_features, dim=-1)
# #             mean, logvar = self.encode(x, split=True)
#         z = self.sampler(mean, logvar)
#         xx = self.decoder(z)

        z, mean, logvar = self.embed(x, return_params=True)
        xx = self.decoder(z)

        return xx, mean, logvar



In [319]:
class KLDivergence(nn.Module):

    def __init__(self, alpha=1.0):
        super().__init__()
        self.alpha = alpha

    def forward(self, mean, logvar):
        kl = -0.5 * torch.sum(1 + logvar - mean.pow(2) - logvar.exp())
        return kl * self.alpha


In [320]:
class MSELoss(nn.Module):

    def __init__(self, alpha=1.0):
        super().__init__()
        self.alpha = alpha

    def forward(self, y_true, y_pred):
        mse = torch.mean((y_true-y_pred)**2)
        return mse * self.alpha

In [321]:
from torch.utils.data import Dataset, DataLoader

In [322]:
class SCRNASeqDataset(Dataset):

    def __init__(self, data):
        self.data = (data / data.max()).astype(np.float32)

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        sample = torch.tensor(np.squeeze(self.data[idx].A).astype(np.float32))

        return sample

In [323]:
def train_epoch(dataloader, model, recon_loss_fn, regular_loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    sample_counter = 0
    for batch, x in enumerate(dataloader):
        optimizer.zero_grad()

        x = x.to(device)

        # Reconstruction and regularization losses
        xx, mean, logvar = model(x)
        regular_loss = regular_loss_fn(mean, logvar)
        recon_loss = recon_loss_fn(x, xx)

        # Backpropagation
        loss = regular_loss + recon_loss
        loss.backward()
        optimizer.step()

        sample_counter += len(x)
        if batch % 1000 == 0:
            loss = loss.item()/len(x)
            regular_loss = regular_loss.item()/len(x)
            recon_loss = recon_loss.item()/len(x)
            
            print(f"[{batch:>5d}/{len(dataloader):>5d}] L: {loss:>7e}  KL: {regular_loss:>7e} MSE: {recon_loss:>7e}")

In [324]:
def test(dataloader, model, recon_loss_fn, regular_loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()

    regular_loss = 0
    recon_loss = 0
    with torch.no_grad():
        for x in dataloader:
            x = x.to(device)

            # Reconstruction and regularization losses
            xx, mean, logvar = model(x)
            regular_loss += regular_loss_fn(mean, logvar)
            recon_loss += recon_loss_fn(x, xx)

    avg_regular = regular_loss / num_batches
    avg_recon = recon_loss / num_batches
    avg_total = avg_regular + avg_recon
    print(f"loss total: {(avg_total):>8e}\t regular: {(avg_regular):>8e}\t recon: {(avg_recon):>8e}\n")

In [331]:
#placeholder random data
gex_dataset = SCRNASeqDataset(gex_feature_filtered.X)

gex_train_dataloader = DataLoader(gex_dataset[:50_000], batch_size=16, shuffle=True)
gex_val_dataloader = DataLoader(gex_dataset[50_000:], batch_size=16, shuffle=True)

In [327]:
model = VarAutoencoder(n_best_features, 32)
model = model.to(device)
summary(model, input_size=(69, n_best_features))

Layer (type:depth-idx)                   Output Shape              Param #
VarAutoencoder                           [69, 100]                 --
├─Sequential: 1-1                        [69, 64]                  --
│    └─Linear: 2-1                       [69, 64]                  6,464
│    └─ReLU: 2-2                         [69, 64]                  --
│    └─Linear: 2-3                       [69, 64]                  4,160
│    └─SplitActivator: 2-4               [69, 64]                  --
│    │    └─Tanh: 3-1                    [69, 32]                  --
│    │    └─ReLU: 3-2                    [69, 32]                  --
├─Sampler: 1-2                           [69, 32]                  --
├─Sequential: 1-3                        [69, 100]                 --
│    └─Linear: 2-5                       [69, 64]                  2,112
│    └─ReLU: 2-6                         [69, 64]                  --
│    └─Linear: 2-7                       [69, 100]                 6,500
│  

In [328]:
regular_loss_fn = KLDivergence(0.1)
recon_loss_fn = MSELoss(1.0)
optimizer = torch.optim.Adam(model.parameters(), lr=5E-3)

In [332]:
num_epochs = 30

for e in range(num_epochs):
    print(f"Epoch {e+1}\n-------------------------------")
    train_epoch(gex_train_dataloader, model, recon_loss_fn, regular_loss_fn, optimizer)
    test(gex_val_dataloader, model, regular_loss_fn, recon_loss_fn)

Epoch 1
-------------------------------
[    0/ 3125] L: 4.412609e-07  KL: 4.261732e-07 MSE: 1.508773e-08
[ 1000/ 3125] L: 4.565187e-07  KL: 2.086163e-07 MSE: 2.479024e-07
[ 2000/ 3125] L: 2.327936e-07  KL: 2.086163e-07 MSE: 2.417732e-08
[ 3000/ 3125] L: 4.089184e-07  KL: 2.145767e-07 MSE: 1.943417e-07
loss total: 3.519783e-05	 regular: 4.267071e-08	 recon: 3.515516e-05

Epoch 2
-------------------------------
[    0/ 3125] L: 3.506046e-07  KL: 2.563000e-07 MSE: 9.430458e-08
[ 1000/ 3125] L: 1.344255e-06  KL: 2.384186e-07 MSE: 1.105836e-06
[ 2000/ 3125] L: 2.299716e-07  KL: 1.907349e-07 MSE: 3.923671e-08
[ 3000/ 3125] L: 4.207043e-07  KL: 2.026558e-07 MSE: 2.180485e-07
loss total: 2.454059e-05	 regular: 7.295990e-09	 recon: 2.453330e-05

Epoch 3
-------------------------------
[    0/ 3125] L: 2.174984e-07  KL: 1.966953e-07 MSE: 2.080302e-08
[ 1000/ 3125] L: 2.605826e-07  KL: 2.205372e-07 MSE: 4.004541e-08
[ 2000/ 3125] L: 2.751903e-07  KL: 2.026558e-07 MSE: 7.253454e-08
[ 3000/ 3125] 

In [350]:
np.random.seed(42)
shuffled_indices = np.arange(gex_feature_filtered.shape[0])
np.random.shuffle(shuffled_indices)

In [351]:
tsne_subset = gex_feature_filtered[shuffled_indices[:1000], :]
y = tsne_subset.obs['cell_type']

In [334]:
tsne_sample = model.embed(torch.tensor(tsne_subset.X.A).to(device)).clamp(min=1E-9, max=1.0).cpu().detach().numpy()

In [353]:
# from sklearn.manifold import TSNE
import plotly.express as px

tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(tsne_sample)
tsne.kl_divergence_

2.357104539871216

In [354]:
fig = px.scatter(x=X_tsne[:, 0], y=X_tsne[:, 1], color=y)
fig.update_layout(
    title="t-SNE visualization of Custom Classification dataset",
    xaxis_title="First t-SNE",
    yaxis_title="Second t-SNE",
)
fig.show()

/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2044: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [361]:
#placeholder random data
atac_dataset = SCRNASeqDataset(gex_feature_filtered.X)

atac_train_dataloader = DataLoader(atac_dataset[:50_000], batch_size=16, shuffle=True)
atac_val_dataloader = DataLoader(atac_dataset[50_000:], batch_size=16, shuffle=True)

In [362]:
atac_dataset.data.shape

(69249, 100)

In [363]:
model2 = VarAutoencoder(n_best_features, 32)
model2 = model2.to(device)
summary(model2, input_size=(69, n_best_features))

Layer (type:depth-idx)                   Output Shape              Param #
VarAutoencoder                           [69, 100]                 --
├─Sequential: 1-1                        [69, 64]                  --
│    └─Linear: 2-1                       [69, 64]                  6,464
│    └─ReLU: 2-2                         [69, 64]                  --
│    └─Linear: 2-3                       [69, 64]                  4,160
│    └─SplitActivator: 2-4               [69, 64]                  --
│    │    └─Tanh: 3-1                    [69, 32]                  --
│    │    └─ReLU: 3-2                    [69, 32]                  --
├─Sampler: 1-2                           [69, 32]                  --
├─Sequential: 1-3                        [69, 100]                 --
│    └─Linear: 2-5                       [69, 64]                  2,112
│    └─ReLU: 2-6                         [69, 64]                  --
│    └─Linear: 2-7                       [69, 100]                 6,500
│  

In [364]:
regular_loss_fn2 = KLDivergence(0.1)
recon_loss_fn2 = MSELoss(1.0)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=5E-3)

In [365]:
num_epochs = 30

for e in range(num_epochs):
    print(f"Epoch {e+1}\n-------------------------------")
    train_epoch(atac_train_dataloader, model2, recon_loss_fn2, regular_loss_fn2, optimizer2)
    test(atac_val_dataloader, model2, regular_loss_fn2, recon_loss_fn2)

Epoch 1
-------------------------------
[    0/ 3125] L: 2.672824e-02  KL: 1.081111e-02 MSE: 1.591714e-02
[ 1000/ 3125] L: 2.821828e-05  KL: 2.777204e-07 MSE: 2.794056e-05
[ 2000/ 3125] L: 7.426662e-07  KL: 1.933426e-07 MSE: 5.493236e-07
[ 3000/ 3125] L: 3.321522e-07  KL: 1.907349e-07 MSE: 1.414174e-07
loss total: 6.430265e-05	 regular: 1.623412e-09	 recon: 6.430103e-05

Epoch 2
-------------------------------
[    0/ 3125] L: 3.015931e-07  KL: 1.907349e-07 MSE: 1.108583e-07
[ 1000/ 3125] L: 2.102032e-07  KL: 1.907349e-07 MSE: 1.946836e-08
[ 2000/ 3125] L: 6.615746e-07  KL: 5.029142e-07 MSE: 1.586604e-07
[ 3000/ 3125] L: 6.000312e-06  KL: 1.266412e-06 MSE: 4.733900e-06
loss total: 2.716880e-05	 regular: 1.616995e-08	 recon: 2.715263e-05

Epoch 3
-------------------------------
[    0/ 3125] L: 2.353646e-07  KL: 2.026558e-07 MSE: 3.270882e-08
[ 1000/ 3125] L: 8.791304e-07  KL: 3.267080e-07 MSE: 5.524224e-07
[ 2000/ 3125] L: 2.749293e-07  KL: 2.004206e-07 MSE: 7.450869e-08
[ 3000/ 3125] 

In [366]:
tsne_subset2 = atac_feature_filtered[shuffled_indices[:1000], :]
y2 = tsne_subset2.obs['cell_type']

In [370]:
tsne_sample2 = model2.embed(torch.tensor(tsne_subset2.X.A).to(device)).clamp(min=1E-9, max=1.0).cpu().detach().numpy()

In [371]:
# from sklearn.manifold import TSNE
import plotly.express as px

tsne2 = TSNE(n_components=2, random_state=42)
X_tsne2 = tsne2.fit_transform(tsne_sample)
tsne2.kl_divergence_

2.357104539871216

In [372]:
fig2 = px.scatter(x=X_tsne2[:, 0], y=X_tsne2[:, 1], color=y2)
fig2.update_layout(
    title="t-SNE visualization of Custom Classification dataset",
    xaxis_title="First t-SNE",
    yaxis_title="Second t-SNE",
)
fig2.show()

/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2044: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

